In [ ]:
import pandas as pd
import os
import gensim
import nltk as nl
from sklearn.linear_model import LogisticRegression


#Reading a csv file with text data
dbFilepandas = pd.read_csv('for_my_love.csv', index_col = 0).apply(lambda x: x.astype(str).str.lower())

train = []
#getting only the first 4 columns of the file 
for sentences in dbFilepandas[dbFilepandas.columns[0:4]].values:
    train.extend(sentences)

# Create an array of tokens using nltk
tokens = [nl.word_tokenize(sentences) for sentences in train]

In [ ]:
len(tokens)

In [ ]:
%%time
model = gensim.models.Word2Vec(tokens, size=300, min_count=1, workers=3)
print("\n Training the word2vec model...\n")
# reducing the epochs will decrease the computation time
model.train(tokens, total_examples=len(tokens), epochs=4000)
# You can save your model if you want....

# The two datasets must be the same size
max_dataset_size = len(model.wv.vectors)

In [ ]:
max_dataset_size

In [ ]:
Y_dataset = []

# get the last number of each file. In this case is the department number
# this will be the 0 or 1, or another kind of classification. ( to use words you need to extract them differently, this way is to numbers)
with open("for_my_love.csv", "r", encoding='utf-8') as f:
    for line in f:
        lastchar = line.strip()[0]
        if lastchar.isdigit():
            result = int(lastchar) 
            Y_dataset.append(result) 
        else:
            result = 40

In [ ]:
len(Y_dataset)

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(model.wv.vectors, Y_dataset)

In [ ]:
# Prediction of the first 15 samples of all features
predict = clf.predict(model.wv.syn0[:15, :])
# Calculating the score of the predictions
score = clf.score(model.wv.syn0, Y_dataset[:max_dataset_size])
print("\nPrediction word2vec : \n", predict)
print("Score word2vec : \n", score)